In [1]:
import sys
sys.path.append('../')

In [2]:
import os
import cv2
import glob
import fiona
from skimage import color
import rasterio
import numpy as np
import rasterio.mask
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from preprocess.argument_utils import data_augment
 
import warnings
warnings.filterwarnings("ignore")

def remove_empty(shp_path):
    for _ in glob.glob(shp_path):
        with fiona.open(_, "r") as shapefile:
            features = [f["geometry"] for f in shapefile]
        if all(x is None for x in features):
            try:
                os.remove(_)
                os.remove(_.replace('shp', 'dbf'))
                os.remove(_.replace('shp', 'prj'))
                os.remove(_.replace('shp', 'shx'))
            except:
                pass
            try:
                extend = _.split('/')[-2].replace('crop_shape','crop')
                os.remove(_.replace('shp', 'tif').replace(_.split('/')[-2], extend))
            except:
                pass
        shapefile.close

class generator_train(utils.Sequence):
    def __init__(self, file_names, batch_size, N_CLASSES, numband):
        self.file_names = file_names
        self.batch_size = batch_size
        self.N_CLASSES = N_CLASSES
        self.numband = numband

    def __len__(self):
        return int(np.floor(len(self.file_names) / self.batch_size))

    def __getitem__(self, index):
        path_mask = self.file_names[index]
        path_img = path_mask.replace('mask_cut_crop','img_cut_crop')
        mask = rasterio.open(path_mask).read().swapaxes(0,1).swapaxes(1,2)
        image = rasterio.open(path_img).read().swapaxes(0,1).swapaxes(1,2)
        image = image[:,:,:self.numband]
        image = tf.cast(image, tf.float32)
        image, mask = data_augment(image, mask[:,:,0], self.N_CLASSES)
        
        _, image_size_height, image_size_wight = image.shape
        lab = color.rgb2lab(image)
        img_edge = cv2.Canny(np.asarray(np.uint8(lab)),0,0)
        
        mask_edge =  cv2.Canny(np.asarray(np.uint8(mask*255)),10,1000)
        return image/255.0, img_edge/255.0, mask, mask_edge/255.0

    def on_epoch_end(self):
        np.random.shuffle(self.file_names)

class generator_valid(utils.Sequence):
    def __init__(self, file_names, batch_size, N_CLASSES, numband):
        self.file_names = file_names
        self.batch_size = batch_size
        self.N_CLASSES = N_CLASSES
        self.numband = numband

    def __len__(self):
        return int(np.floor(len(self.file_names) / self.batch_size))

    def __getitem__(self, index):
        path_mask = self.file_names[index]
        path_img = path_mask.replace('mask_cut_crop','img_cut_crop')
        mask = rasterio.open(path_mask).read().swapaxes(0,1).swapaxes(1,2)
        image = rasterio.open(path_img).read().swapaxes(0,1).swapaxes(1,2)
        image = image[:,:,:self.numband]
        image = tf.cast(image, tf.float32)
        mask = tf.cast(mask, tf.uint8)
        mask = tf.one_hot(mask[:,:,0], depth=self.N_CLASSES)
        
        _, image_size_height, image_size_wight = image.shape
        lab = color.rgb2lab(image)
        img_edge = cv2.Canny(np.asarray(np.uint8(lab)),0,0)
        
        mask_edge =  cv2.Canny(np.asarray(np.uint8(mask*255)),10,1000)
        return image/255.0, img_edge/255.0, mask, mask_edge/255.0

    def on_epoch_end(self):
        np.random.shuffle(self.file_names)
        
def input_fn(file_names, image_size, batch_size, N_CLASSES, numband ,train=True):
    def generator_fn():
        if train:
            generator = utils.OrderedEnqueuer(generator_train(file_names, batch_size, N_CLASSES, numband), False)
        else:
            generator = utils.OrderedEnqueuer(generator_valid(file_names, batch_size, N_CLASSES, numband), False)
        generator.start()
        
        n = 0
        while n<int(len(file_names)):
            image, img_edge, mask, mask_edge = generator.get().__next__()
            yield ((image, img_edge), (mask, mask_edge, mask, mask))
            n+=1

    output_types = ((tf.float32, tf.float32), (tf.float32, tf.float32, tf.float32, tf.float32))
    output_shapes = (((image_size, image_size, numband),
                     (image_size, image_size)),
                     ((image_size, image_size, N_CLASSES),
                    (image_size, image_size),
                     (image_size, image_size, N_CLASSES),
                     (image_size, image_size, N_CLASSES)),)

    dataset = tf.data.Dataset.from_generator(generator=generator_fn,
                                             output_types=output_types,
                                             output_shapes=output_shapes)
    dataset = dataset.repeat(1)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def data_gen(mask_path, img_size, batch_size, N_CLASSES, numband, split_ratios, test_data=False, multi=False):
    # print(mask_path)
    mask_img = sorted(glob.glob(mask_path))
    np.random.shuffle(mask_img)
    L = len(mask_img)
    if multi:
        if test_data:
            L_train = int(split_ratios*L)
            L_valid = int((1-split_ratios)/2*L)
            L_test = L - L_train - L_valid
            train_dataset = input_fn_multi(mask_img[:L_train], img_size, batch_size, N_CLASSES, numband)
            valid_dataset = input_fn_multi(mask_img[L_train:L_train+L_valid], img_size, batch_size, N_CLASSES, numband, False)
            test_dataset = input_fn_multi(mask_img[L_train+L_valid:L], img_size, batch_size, N_CLASSES, numband, False)
            print("Training:validation:testing = {}:{}:{}".format(L_train, L_valid, L_test))
            return train_dataset, valid_dataset, test_dataset, L_train, L_valid, L_test
        else:
            L_train = int(split_ratios*L)
            L_valid = int(L-L_train)
            train_dataset = input_fn_multi(mask_img[:L_train], img_size, batch_size, N_CLASSES, numband)
            valid_dataset = input_fn_multi(mask_img[L_train:L], img_size, batch_size, N_CLASSES, numband, False)
            print("Training:validation = {}:{}".format(L_train, L_valid))
        return train_dataset, valid_dataset, L_train, L_valid
    else:
        if test_data:
            L_train = int(split_ratios*L)
            L_valid = int((1-split_ratios)/2*L)
            L_test = L - L_train - L_valid
            train_dataset = input_fn(mask_img[:L_train], img_size, batch_size, N_CLASSES, numband)
            valid_dataset = input_fn(mask_img[L_train:L_train+L_valid], img_size, batch_size, N_CLASSES, numband, False)
            test_dataset = input_fn(mask_img[L_train+L_valid:L], img_size, batch_size, N_CLASSES, numband, False)
            print("Training:validation:testing = {}:{}:{}".format(L_train, L_valid, L_test))
            return train_dataset, valid_dataset, test_dataset, L_train, L_valid, L_test
        else:
            L_train = int(split_ratios*L)
            L_valid = int(L-L_train)
            train_dataset = input_fn(mask_img[:L_train], img_size, batch_size, N_CLASSES, numband)
            if L_train == L:
                valid_dataset = input_fn(mask_img[int(L_train*0.8):L], img_size, batch_size, N_CLASSES, numband, False)
                print("Training:validation = {}:{}".format(L_train, L-int(L_train*0.8)))
            else:
                valid_dataset = input_fn(mask_img[L_train:L], img_size, batch_size, N_CLASSES, numband, False)
                print("Training:validation = {}:{}".format(L_train, L_valid))
        return train_dataset, valid_dataset, L_train, L_valid

In [3]:
out_path = '/home/quyet/DATA_ML/WorkSpace/segmentation/data/road_multi'
overlap_mask = os.path.join(out_path, 'mask_cut_crop')
train_dataset, valid_dataset, _, _ = data_gen(os.path.join(overlap_mask, '*.tif'), img_size=256, 
                                                            batch_size=2, N_CLASSES=1, numband=3, 
                                                            split_ratios=0.8, test_data=False, multi=False)

Training:validation = 1804:452


2022-07-12 09:16:00.679616: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4795 MB memory:  -> device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:05:00.0, compute capability: 6.1


In [4]:
import os
from PIL import Image
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from PIL import Image
np.random.seed(123)
import warnings
import keras
from keras.layers import Dense, Dropout,Input,Average,Conv2DTranspose,SeparableConv2D,dot,UpSampling2D,Add, Flatten,Concatenate,Multiply,Conv2D, MaxPooling2D,Activation,AveragePooling2D, ZeroPadding2D,GlobalAveragePooling2D,multiply,DepthwiseConv2D,ZeroPadding2D,GlobalAveragePooling2D,concatenate ,Lambda
from keras.initializers import RandomNormal

from keras import backend as K
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import numpy as np
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from tqdm import tqdm_notebook as tqdm
import cv2
from sklearn.utils import shuffle
import tifffile as tif
from sklearn.model_selection import train_test_split
# from keras.optimizers import Adam, Nadam
from glob import glob
from sklearn.utils import shuffle
import skimage.io
from skimage.transform import rescale, resize, downscale_local_mean
from skimage.color import rgb2gray
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=1.0)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
# sess = tf.compat.v1.Session()

def spatial_att_block(x,intermediate_channels):
    out = Conv2D(intermediate_channels,kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(x)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = Conv2D(1,kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(out)
    out = Activation('sigmoid')(out)
    return out
    
def resblock(x,ip_channels,op_channels,stride=(1,1)):
    residual = x
    out = Conv2D(op_channels,kernel_size=(3,3),strides=stride,padding='same', kernel_initializer = 'he_normal')(x)
    out = BatchNormalization()(out)
    out = Activation('relu')(out)
    out = Conv2D(op_channels,kernel_size=(3,3),strides=stride,padding='same', kernel_initializer = 'he_normal')(x)
    out = BatchNormalization()(out)
    out = Add()([out,residual])
    out = Activation('relu')(out)
    return out

def dual_att_blocks(skip,prev,out_channels):
    up = Conv2DTranspose(out_channels,4, strides=(2, 2), padding='same')(prev)
    up = BatchNormalization()(up)
    up = Activation('relu')(up)
    inp_layer = Concatenate()([skip,up])
    inp_layer = Conv2D(out_channels,3,strides=(1,1),padding='same', kernel_initializer = 'he_normal')(inp_layer)
    inp_layer = BatchNormalization()(inp_layer)
    inp_layer = Activation('relu')(inp_layer)
    se_out = se_block(inp_layer,out_channels)
    sab = spatial_att_block(inp_layer,out_channels//4)
    #sab = Add()([sab,1])
    sab = Lambda(lambda y : y+1)(sab)
    final = Multiply()([sab,se_out])
    return final
    

def gsc(input_features,gating_features,in_channels,out_channels,kernel_size=1,stride=1,dilation=1,groups=1):
    x = Concatenate()([input_features,gating_features])
    x = BatchNormalization()(x)
    x = Conv2D(in_channels+1, (1,1), strides =(1,1), padding='same', kernel_initializer = 'he_normal')(x)
    x = Activation('relu')(x)
    x = Conv2D(1,kernel_size=(1,1),strides=1,padding='same', kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('sigmoid')(x)
    return x

def se_block(in_block, ch, ratio=16):
    x = GlobalAveragePooling2D()(in_block)
    x = Dense(ch//ratio, activation='relu')(x)
    x = Dense(ch, activation='sigmoid')(x)
    return Multiply()([in_block, x])

def Attention_B(X, G, k):
    FL = int(X.shape[-1])
    init = RandomNormal(stddev=0.02)
    theta = Conv2D(k,(2,2), strides = (2,2), padding='same')(X)
    Phi = Conv2D(k, (1,1), strides =(1,1), padding='same', use_bias=True)(G)
   
    ADD = Add()([theta, Phi])
    ADD = Activation('relu')(ADD)
    Psi = Conv2D(1,(1,1), strides = (1,1), padding="same",kernel_initializer=init)(ADD)
    Psi = Activation('sigmoid')(Psi)
    Up = Conv2DTranspose(1, (2,2), strides=(2, 2), padding='valid')(Psi)
    Final = Multiply()([X, Up])
    Final = BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-5)(Final)
    print(Final.shape)
    return Final
def Unet3(input_shape,n_filters,kernel=(3,3),strides=(1,1),pad='same'):
    x = input_shape
    conv1 = Conv2D(n_filters,kernel_size=kernel,strides=strides,padding=pad, kernel_initializer = 'he_normal')(input_shape)
    conv1 = BatchNormalization(axis=-1)(conv1)
    conv1 = LeakyReLU(alpha=0.1)(conv1)
   
    conv2 = Conv2D(n_filters,kernel_size=kernel,strides=strides,padding=pad, kernel_initializer = 'he_normal')(conv1)
    conv2 = BatchNormalization(axis=-1)(conv2)
    conv2 =  LeakyReLU(alpha=0.1)(conv2)
   
    x = Conv2D(n_filters,kernel_size = (1,1),strides = (1,1),padding = 'same', kernel_initializer = 'he_normal')(x)
   
    return Add()([x,conv2])
def Up3(input1,input2,kernel=(3,3),stride=(1,1), pad='same'):
    #up = UpSampling2D(2)(input2)
    up = Conv2DTranspose(int(input1.shape[-1]),(1, 1), strides=(2, 2), padding='same', kernel_initializer = 'he_normal')(input2)
    up = Concatenate()([up,input1])
    
    #up1 = BatchNormalization()(up)
    #up1 =  LeakyReLU(alpha=0.25)(up1)
    #up1 = Conv2D(int(input1.shape[-1]),kernel_size=(3,3),strides=(1,1),padding='same')(up1)
    #up1 = BatchNormalization()(up1)
    #up1 =  LeakyReLU(alpha=0.25)(up1)
    #up1 = Conv2D(int(input1.shape[-1]),kernel_size=(3,3),strides=(1,1),padding='same')(up1)
    #up2 = Add()([up1,up])
    return up
    
    return Unet3(up,int(input1.shape[-1]),kernel,stride,pad)
def gatingSig(input_shape,n_filters,kernel=(1,1),strides=(1,1),pad='same'):
    conv = Conv2D(n_filters,kernel_size=kernel,strides=strides,padding=pad, kernel_initializer = 'he_normal')(input_shape)
    conv = BatchNormalization(axis=-1)(conv)
    return LeakyReLU(alpha=0.1)(conv)

def DSup(x, var):
    d = Conv2D(1,(1,1), strides=(1,1), padding = "same", kernel_initializer = 'he_normal')(x)
    d = UpSampling2D(var)(d)
    return d

def DSup1(x, var):
    d = Conv2D(1,(2,2), strides=(2,2), padding = "same", kernel_initializer = 'he_normal')(x)
    d = UpSampling2D(var)(d)
    return d

#Keras


def msrf(input_size=(512,512,3),input_size_2=(512,512,1)):
    n_labels=1
    feature_scale=8
    #input_shape= Image.shape
    filters = [64, 128, 256, 512,1024]
    atrous_rates = (6, 12, 18)
    n_labels=1
    feature_scale=8
    #input_shape= Image.shape
    filters = [64, 128, 256, 512,1024]

    inputs_img = Input(input_size)
    inputs_img = tf.image.resize(inputs_img, size=(256,256))
    canny = Input(input_size_2, name='checkdim')
    canny = tf.image.resize(canny, size=(256,256))
    n11 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs_img)
    n11 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n11)
    n11 = BatchNormalization()(n11)
    n11 = se_block(n11,32)

    
    n12 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n11)
    n12 = BatchNormalization()(n12)
    #here
    n12 = Add()([n12,n11])
    pred1 = Conv2D(1,(1,1), strides=(1,1), padding="same",activation='sigmoid', kernel_initializer = 'he_normal')(n12)
    
    pool1 = MaxPooling2D(pool_size=(2, 2))(n11)
    pool1 = Dropout(0.2)(pool1)
    n21 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    n21 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n21)
    n21 = BatchNormalization()(n21)
    n21 = se_block(n21,64)
    #here
    
    
    n22 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n21)
    n22 = BatchNormalization()(n22)
    n22 = Add()([n22,n21])
    
    pool2 = MaxPooling2D(pool_size=(2, 2))(n21)
    pool2 = Dropout(0.2)(pool2)
    
    n31 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    n31 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n31)
    n31 = BatchNormalization()(n31)
    n31 = se_block(n31,128)
   
    
    n32 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n31)
    n32 = BatchNormalization()(n32)
    n32 = Add()([n32,n31])
    
    pool3 = MaxPooling2D(pool_size=(2, 2))(n31)
    pool3 = Dropout(0.2)(pool3)
    n41 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    n41 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n41)
    n41 = BatchNormalization()(n41)
    #############################################ASPP
    shape_before = n41.shape
    
    
    
    #############################################
    
    n12,n22 = RDDB(n11,n21,32,64,16)
    # pred2 = Conv2D(1,(1,1), strides=(1,1), padding="same",activation='sigmoid')(n12)
    
    n32,n42 = RDDB(n31,n41,128,256,64)
    
    n12,n22 = RDDB(n12,n22,32,64,16)
    # pred3 = Conv2D(1,(1,1), strides=(1,1), padding="same",activation='sigmoid')(n12)
    
    n32,n42 = RDDB(n32,n42,128,256,64)
    
    n22,n32 = RDDB(n22,n32,64,128,32)
    
    
    n13,n23 = RDDB(n12,n22,32,64,16)
    
    n33,n43 = RDDB(n32,n42,128,256,64)
    
    n23,n33 = RDDB(n23,n33,64,128,32)
    
    n13,n23 = RDDB(n12,n22,32,64,16)
    
    n33,n43 = RDDB(n32,n42,128,256,64)
    
    n13 = Lambda(lambda x: x * 0.4)(n13)
    n23 = Lambda(lambda x: x * 0.4)(n23)
    n33 = Lambda(lambda x: x * 0.4)(n33)
    n43 = Lambda(lambda x: x * 0.4)(n43)
    
    
    n13,n23 = Add()([n11,n13]),Add()([n21,n23])
    n33,n43 = Add()([n31,n33]),Add()([n41,n43])

    ###############Shape Stream

    
    d0 = Conv2D(32,kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(n23)
    ss = keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(d0)
    ss = resblock(ss,32,32)
    c3 = Conv2D(1, kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(n33)
    c3 = keras.layers.UpSampling2D(size=(4, 4), data_format=None, interpolation='bilinear')(c3)
    ss = Conv2D(16,kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(ss)
    ss = gsc(ss,c3,32,32)
    ss = resblock(ss,16,16)
    ss = Conv2D(8,kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(ss)
    c4 = Conv2D(1, kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(n43)
    c4 = keras.layers.UpSampling2D(size=(8, 8), data_format=None, interpolation='bilinear')(c4)
    ss = gsc(ss,c4,16,16)
    ss = resblock(ss,8,8)
    ss = Conv2D(4,kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(ss)
    ss = Conv2D(1,kernel_size=(1,1),padding='same', kernel_initializer = 'he_normal')(ss)
    edge_out = Activation('sigmoid',name='edge_out')(ss)

    #######canny edge
    # canny = cv2.Canny(np.asarray(canny),10,100)
    cat = Concatenate()([edge_out,canny])
    cw = Conv2D(1,kernel_size=(1,1),padding='same', kernel_initializer = 'he_normal')(cat)
    acts = Activation('sigmoid')(cw)
    edge = Conv2D(1, kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(acts)
    edge = BatchNormalization()(edge)
    edge = Activation('relu')(edge)
    
    
    #########################
    
   
    n34_preinput=Attention_B(n33,n43,128)
    n34 = Up3(n34_preinput,n43)
    n34_d = dual_att_blocks(n33,n43,128)
    n34_t = Concatenate()([n34,n34_d])
    n34_t = Conv2D(128,kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(n34_t)
    n34_2 = BatchNormalization()(n34_t)
    n34_2 = Activation('relu')(n34_2)
    n34_2 = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(n34_2)
    n34_2 = BatchNormalization()(n34_2)
    n34_2 = Activation('relu')(n34_2)
    n34_2 = Conv2D(128,kernel_size=(3,3),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(n34_2)
    n34 = Add()([n34_2,n34_t])
    pred4 = Conv2D(1,kernel_size=(1,1),strides=(1,1),padding='same',activation="sigmoid", kernel_initializer = 'he_normal')(n34)
    pred4 = UpSampling2D(size=(4,4),interpolation='bilinear',name='pred4')(pred4)

    
   
    n24_preinput =Attention_B(n23,n34,64)
    n24 = Up3(n24_preinput,n34)
    n24_d = dual_att_blocks(n23,n34,64)
    n24_t = Concatenate()([n24,n24_d])
    n24_t = Conv2D(64,kernel_size=(1,1),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(n24_t)
    n24_2 = BatchNormalization()(n24_t)
    n24_2 = Activation('relu')(n24_2)
    n24_2 = Conv2D(64,kernel_size=(3,3),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(n24_2)
    n24_2 = BatchNormalization()(n24_2)
    n24_2 = Activation('relu')(n24_2)
    n24_2 = Conv2D(64,kernel_size=(3,3),strides=(1,1),padding='same', kernel_initializer = 'he_normal')(n24_2)
    n24 = Add()([n24_2,n24_t])
    pred2 = Conv2D(1,kernel_size=(1,1),strides=(1,1),padding="same" , activation="sigmoid", kernel_initializer = 'he_normal')(n24)
    pred2 = UpSampling2D(size=(2,2),interpolation='bilinear',name='pred2')(pred2)
   
    n14_preinput = Conv2DTranspose(32,4, strides=(2, 2), padding='same')(n24)
    n14_input = Concatenate()([n14_preinput,n13])
    n14_input = Conv2D(32, 1, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n14_input)
    n14 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n14_input)
    n14 = BatchNormalization()(n14)
    n14 = Add()([n14,n14_input])
    n14 = Concatenate()([n14,edge])
    n14 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(n14)
    n14 = BatchNormalization()(n14)
    x = Conv2D(1,(1,1), strides=(1,1), padding="same",activation='sigmoid',name='x', kernel_initializer = 'he_normal')(n14)
    
    model = Model(inputs= [inputs_img,canny],outputs = [x, edge_out, pred2, pred4])
    return model

def RDDB(x,y,nf1=128,nf2=1212,gc=64,bias=True):
    x1 = Conv2D(filters=gc, kernel_size=3, strides=1,padding='same', use_bias=bias, kernel_initializer = 'he_normal')(x)
    x1 = LeakyReLU(alpha=0.25)(x1)
    
    y1 = Conv2D(filters=gc, kernel_size=3, strides=1,padding='same', use_bias=bias, kernel_initializer = 'he_normal')(y)
    y1 = LeakyReLU(alpha=0.25)(y)
    
    x1c = Conv2D(filters=gc, kernel_size=3, strides=2,padding='same', use_bias=bias, kernel_initializer = 'he_normal')(x)
    x1c = LeakyReLU(alpha=0.25)(x1c)
    y1t = Conv2DTranspose(filters=gc, kernel_size=3, strides=2,padding='same', use_bias=bias)(y)
    y1t = LeakyReLU(alpha=0.25)(y1t)
    
    
    x2_input = concatenate([x,x1,y1t],axis=-1)
    x2 = Conv2D(filters= gc, kernel_size=3,strides=1, padding='same',use_bias=bias, kernel_initializer = 'he_normal')(x2_input)
    x2 = LeakyReLU(alpha=0.25)(x2)
    
    y2_input = concatenate([y,y1,x1c],axis=-1)
    y2 = Conv2D(filters= gc, kernel_size=3,strides=1, padding='same',use_bias=bias, kernel_initializer = 'he_normal')(y2_input)
    y2 = LeakyReLU(alpha=0.25)(y2)
    
    x2c = Conv2D(filters=gc, kernel_size=3, strides=2,padding='same', use_bias=bias, kernel_initializer = 'he_normal')(x1)
    x2c = LeakyReLU(alpha=0.25)(x2c)
    y2t = Conv2DTranspose(filters=gc, kernel_size=3, strides=2,padding='same', use_bias=bias)(y1)
    y2t = LeakyReLU(alpha=0.25)(y2t)
    
    
    
    x3_input = concatenate([x,x1,x2,y2t] , axis=-1)
    x3 = Conv2D(filters= gc, kernel_size=3,strides=1, padding='same', use_bias=bias, kernel_initializer = 'he_normal')(x3_input)
    x3 = LeakyReLU(alpha=0.25)(x3)
    
    y3_input = concatenate([y,y1,y2,x2c] , axis=-1)
    y3 = Conv2D(filters= gc, kernel_size=3,strides=1, padding='same', use_bias=bias, kernel_initializer = 'he_normal')(y3_input)
    y3 = LeakyReLU(alpha=0.25)(y3)
    
    x3c = Conv2D(filters=gc, kernel_size=3, strides=2,padding='same', use_bias=bias, kernel_initializer = 'he_normal')(x3)
    x3c = LeakyReLU(alpha=0.25)(x3c)
    y3t = Conv2DTranspose(filters=gc, kernel_size=3, strides=2,padding='same', use_bias=bias, kernel_initializer = 'he_normal')(y3)
    y3t = LeakyReLU(alpha=0.25)(y3t)
    
    
        
    x4_input = concatenate([x,x1,x2,x3,y3t] , axis=-1)
    x4 = Conv2D(filters= gc, kernel_size=3,strides=1, padding='same', use_bias=bias, kernel_initializer = 'he_normal')(x4_input)
    x4 = LeakyReLU(alpha=0.25)(x4)
    
    
    y4_input = concatenate([y,y1,y2,y3,x3c] , axis=-1)
    y4 = Conv2D(filters= gc, kernel_size=3,strides=1, padding='same', use_bias=bias, kernel_initializer = 'he_normal')(y4_input)
    y4 = LeakyReLU(alpha=0.25)(y4)
    
    x4c = Conv2D(filters=gc, kernel_size=3, strides=2,padding='same', use_bias=bias, kernel_initializer = 'he_normal')(x4)
    x4c = LeakyReLU(alpha=0.25)(x4c)
    y4t = Conv2DTranspose(filters=gc, kernel_size=3, strides=2,padding='same', use_bias=bias)(y4)
    y4t = LeakyReLU(alpha=0.25)(y4t)
    
        
    x5_input = concatenate([x,x1,x2,x3,x4,y4t] , axis=-1)
    x5 = Conv2D(filters= nf1, kernel_size=3,strides=1, padding='same', use_bias=bias, kernel_initializer = 'he_normal')(x5_input)
    x5 = LeakyReLU(alpha=0.25)(x5)
    
    y5_input = concatenate([y,y1,y2,y3,y4,x4c] , axis=-1)
    y5 = Conv2D(filters= nf2, kernel_size=3,strides=1, padding='same', use_bias=bias, kernel_initializer = 'he_normal')(y5_input)
    y5 = LeakyReLU(alpha=0.25)(y5)
        
    x5 = Lambda(lambda x: x * 0.4)(x5)
    y5 = Lambda(lambda x: x * 0.4)(y5)
        
    return Add()([x5,x]),Add()([y5,y])

In [5]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true[:,:,:,0])
    y_pred_f = K.flatten(y_pred[:,:,:,0])
    intersection = K.sum(y_true_f * y_pred_f)
    d1 =  (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return d1

def dice_coefficient_loss(y_true, y_pred):
    return 1.-dice_coefficient(y_true, y_pred)

def seg_loss(y_true, y_pred):
    # print(y_true.shape, y_pred.shape)
    dice_s = dice_coefficient_loss(y_true,y_pred)
    
    #ce_loss = BinaryCrossentropy(y_true,y_pred)
    ce_loss =tf.keras.backend.binary_crossentropy(y_true,y_pred)
    
    return ce_loss + dice_s

def _to_tensor(x, dtype):
    x = tf.convert_to_tensor(x)
    if x.dtype != dtype:
        x = tf.cast(x, dtype)
    return x

def balanced_cross_entropy_loss(y_true, y_pred):
    _epsilon = _to_tensor(tf.keras.backend.epsilon(), y_pred.dtype )
    y_pred = tf.clip_by_value(y_pred, _epsilon, 1 - _epsilon)
    y_pred = tf.math.log(y_pred/ (1 - y_pred))

    y_true = tf.cast(y_true, tf.float32)
    count_neg = tf.reduce_sum(input_tensor=1. - y_true)
    count_pos = tf.reduce_sum(input_tensor=y_true)
    beta = count_neg / (count_neg + count_pos)
    pos_weight = beta / (1 - beta)
    cost = tf.nn.weighted_cross_entropy_with_logits(logits=y_pred, labels=y_true, pos_weight=pos_weight)
    cost = tf.reduce_mean(input_tensor=cost * (1 - beta))
    return tf.compat.v1.where(tf.equal(count_pos, 0.0), 0.0, cost)

G = msrf()
optimizer = Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
G.compile(optimizer = optimizer, loss = {'x':seg_loss,'edge_out':'binary_crossentropy','pred4':seg_loss,'pred2':seg_loss},
            loss_weights={'x':2.,'edge_out':1.,'pred4':1. , 'pred2':1.})

(None, 64, 64, 128)
(None, 128, 128, 64)


In [6]:
from models.callback.save_best import SavebestweightsandEarlyStopping

model_name = 'msrfnet'
mission = 'road'
img_size = 256
num_class = 1 
batch_size = 2

def lr_decay(epoch):
    initial_learningrate=1e-3
    if epoch < 1:
        return initial_learningrate
    else:
        return initial_learningrate * 0.9 ** (epoch)

if batch_size >1:
    val_batch_size = int(batch_size/2)
else:
    val_batch_size = batch_size

checkpoint_filepath= '/home/quyet/DATA_ML/Projects/segmentation/logs/tmp'
log_dir = '/home/quyet/DATA_ML/Projects/segmentation/logs/graph'
weights_path = '/home/quyet/DATA_ML/WorkSpace/segmentation/weights/%s/'%(model_name) +model_name+'_'+mission+'_'+str(img_size)+'_'+str(num_class)+'class.h5'
patience = 10
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only= True, 
                                                                monitor='val_loss', mode='min', save_best_only=True)
model_lrscheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=1)
model_lrreduce_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience, min_lr=1e-7, verbose=1)
model_earlystopping_callback = SavebestweightsandEarlyStopping(patience=patience, weights_path=weights_path)
model_endtrainnan_callback = tf.keras.callbacks.TerminateOnNaN()
model_tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)
model_callbacks = [model_checkpoint_callback, model_lrscheduler_callback,
                    model_lrreduce_callback, model_earlystopping_callback,
                    model_tensorboard_callback,]

# G.load_weights('/home/quyet/DATA_ML/WorkSpace/segmentation/weights/msrfnet/msrfnet_road_256_1class_train.h5')
history_train = G.fit(train_dataset, batch_size=batch_size, epochs=100, verbose=1, 
                      callbacks=model_callbacks, validation_data=valid_dataset, 
                      validation_batch_size=val_batch_size, use_multiprocessing=True)


Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/100


2022-07-12 09:16:34.390202: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-07-12 09:16:34.797408: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-07-12 09:16:37.563435: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.23GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-07-12 09:16:38.861241: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.09GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean 

    902/Unknown - 1041s 1s/step - loss: 2.8677 - x_loss: 0.6840 - edge_out_loss: 0.0950 - pred2_loss: 0.6315 - pred4_loss: 0.7733
Save best train weights.
Save best val weights.


2022-07-12 09:37:36.525927: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 424673280 exceeds 10% of free system memory.
2022-07-12 09:37:38.241688: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 424673280 exceeds 10% of free system memory.
2022-07-12 09:37:39.186315: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 424673280 exceeds 10% of free system memory.


902/902 [==============================] - 1294s 1s/step - loss: 2.8677 - x_loss: 0.6840 - edge_out_loss: 0.0950 - pred2_loss: 0.6315 - pred4_loss: 0.7733 - val_loss: 3.5335 - val_x_loss: 0.7701 - val_edge_out_loss: 0.0622 - val_pred2_loss: 1.0376 - val_pred4_loss: 0.8935 - lr: 0.0010

Epoch 00002: LearningRateScheduler setting learning rate to 0.0009000000000000001.
Epoch 2/100
902/902 [==============================] - ETA: 0s - loss: 2.6569 - x_loss: 0.6192 - edge_out_loss: 0.0542 - pred2_loss: 0.5839 - pred4_loss: 0.7804
Save best train weights.
Save best val weights.


2022-07-12 09:58:25.521320: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 424673280 exceeds 10% of free system memory.
2022-07-12 09:58:27.398365: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 424673280 exceeds 10% of free system memory.


902/902 [==============================] - 1249s 1s/step - loss: 2.6569 - x_loss: 0.6192 - edge_out_loss: 0.0542 - pred2_loss: 0.5839 - pred4_loss: 0.7804 - val_loss: 3.0709 - val_x_loss: 0.7037 - val_edge_out_loss: 0.0561 - val_pred2_loss: 0.7364 - val_pred4_loss: 0.8710 - lr: 9.0000e-04

Epoch 00003: LearningRateScheduler setting learning rate to 0.0008100000000000001.
Epoch 3/100
902/902 [==============================] - ETA: 0s - loss: 2.5240 - x_loss: 0.5878 - edge_out_loss: 0.0510 - pred2_loss: 0.5389 - pred4_loss: 0.7585
Save best train weights.
Save best val weights.
902/902 [==============================] - 1264s 1s/step - loss: 2.5240 - x_loss: 0.5878 - edge_out_loss: 0.0510 - pred2_loss: 0.5389 - pred4_loss: 0.7585 - val_loss: 2.6146 - val_x_loss: 0.6256 - val_edge_out_loss: 0.0559 - val_pred2_loss: 0.5548 - val_pred4_loss: 0.7527 - lr: 8.1000e-04

Epoch 00004: LearningRateScheduler setting learning rate to 0.0007290000000000002.
Epoch 4/100
902/902 [======================

In [50]:
import cv2
import numpy as np

from tqdm import tqdm
from osgeo import gdal
from postprocess.convert_tif import dilation_obj, remove_small_items, write_image

def get_im_by_coord(org_im, start_x, start_y,num_band, padding, crop_size, input_size):
    startx = start_x-padding
    endx = start_x+crop_size+padding
    starty = start_y - padding
    endy = start_y+crop_size+padding
    result=[]
    img = org_im[starty:endy, startx:endx]
    img = img.swapaxes(2,1).swapaxes(1,0)
    for chan_i in range(num_band):
        result.append(cv2.resize(img[chan_i],(input_size, input_size), interpolation = cv2.INTER_CUBIC))
    return np.array(result).swapaxes(0,1).swapaxes(1,2)

def get_img_coords(w, h, padding, crop_size):
    new_w = w + 2*padding
    new_h = h + 2*padding
    cut_w = list(range(padding, new_w - padding, crop_size))
    cut_h = list(range(padding, new_h - padding, crop_size))

    list_hight = []
    list_weight = []
    for i in cut_h:
        if i < new_h - padding - crop_size:
            list_hight.append(i)
    list_hight.append(new_h-crop_size-padding)

    for i in cut_w:
        if i < new_w - crop_size - padding:
            list_weight.append(i)
    list_weight.append(new_w-crop_size-padding)

    img_coords = []
    for i in list_weight:
        for j in list_hight:
            img_coords.append([i, j])
    return img_coords

def padded_for_org_img(values, num_band, padding):
    padded_org_im = []
    for i in range(num_band):
        band = np.pad(values[i], padding, mode='reflect')
        padded_org_im.append(band)

    values = np.array(padded_org_im).swapaxes(0,1).swapaxes(1,2)
    print(values.shape)
    del padded_org_im
    return values

def predict(model, values, img_coords, num_band, h, w, padding, crop_size, 
            input_size, batch_size, thresh_hold, choose_stage):
    cut_imgs = []
    for i in range(len(img_coords)):
        im = get_im_by_coord(values, img_coords[i][0], img_coords[i][1],
                            num_band,padding, crop_size, input_size)
        cut_imgs.append(im)

    a = list(range(0, len(cut_imgs), batch_size))

    if a[len(a)-1] != len(cut_imgs):
        a[len(a)-1] = len(cut_imgs)

    y_pred = []
    for i in tqdm(range(len(a)-1)):
        x_batch = []
        x_batch = np.array(cut_imgs[a[i]:a[i+1]])
        # print(x_batch.shape)
        img_edge = []
        for img_x in x_batch:
            lab_batch = color.rgb2lab(img_x)  
            img_edge.append(cv2.Canny(np.asarray(np.uint8(lab_batch)),0,0)[..., np.newaxis])
        # print(img_edge.shape)
        img_edge = np.array(img_edge)
        
        # print(x_batch.shape, img_edge.shape)
        y_batch = model.predict((x_batch/255, img_edge/255))
        if len(model.outputs)>1:
            y_batch = y_batch[choose_stage]
        mutilabel = False
        if y_batch.shape[-1]>=2:
            mutilabel = True
            y_batch = np.argmax(y_batch, axis=-1)
        # print(np.unique(y_batch), y_batch.shape)
            
        y_pred.extend(y_batch)
    big_mask = np.zeros((h, w)).astype(np.float16)
    for i in range(len(cut_imgs)):
        true_mask = y_pred[i].reshape((input_size,input_size))
        if not mutilabel:
            true_mask = (true_mask>thresh_hold).astype(np.uint8)
            true_mask = (cv2.resize(true_mask,(input_size, input_size), interpolation = cv2.INTER_CUBIC)>thresh_hold).astype(np.uint8)
            # true_mask = true_mask.astype(np.float16)
        start_x = img_coords[i][1]
        start_y = img_coords[i][0]
        big_mask[start_x-padding:start_x-padding+crop_size, start_y-padding:start_y -
                    padding+crop_size] = true_mask[padding:padding+crop_size, padding:padding+crop_size]
    del cut_imgs
    return big_mask

img_size = 256
num_band = 3
crop_size = 200
batch_size = 1
thresh_hold = 0.8
choose_stage = 0

G.load_weights('/home/quyet/DATA_ML/WorkSpace/segmentation/weights/msrfnet/msrfnet_road_256_1class_val.h5')
image_path = '/home/quyet/DATA_ML/Projects/road_multi/crop/img/test.tif'
dataset = gdal.Open(image_path)
values = dataset.ReadAsArray()[0:num_band]
h,w = values.shape[1:3]    
padding = int((img_size - crop_size)/2)
img_coords = get_img_coords(w, h, padding, crop_size)
values = padded_for_org_img(values, num_band, padding)
big_mask = predict(G, values, img_coords, num_band, h, w, padding, crop_size, 
                    img_size, batch_size, thresh_hold, choose_stage)

(6237, 6126, 3)


100%|█████████████████████████████████████████| 960/960 [03:06<00:00,  5.14it/s]


In [51]:
image_path = '/home/quyet/DATA_ML/Projects/road_multi/crop/img/test.tif'
result_path = write_image(image_path, big_mask)

Write image...
